# Classification

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.optimize as opt

%matplotlib inline

## The data set

We will predict the incidence of diabetes based on various measurements (see [description](https://archive.ics.uci.edu/ml/datasets/Pima+Indians+Diabetes)). Instead of directly using the raw data, we use a normalised version, where the label to be predicted (the incidence of diabetes) is in the first column. Download the data from [the course website](https://machlearn.gitlab.io/isml2017/tutorial/diabetes_scale.csv).

Read in the data using ```np.loadtxt```.

In [2]:
# Solution
names = ['diabetes', 'num preg', 'plasma', 'bp', 'skin fold', 'insulin', 'bmi', 'pedigree', 'age']
data = np.loadtxt('diabetes_scale.csv', delimiter=',')
# Replace -1 with 0 because we need labels to be in {0, 1}
idx = np.where(data[:,0] < 0)
data[idx,0] = 0
data[:5,:]

array([[ 1.        ,  0.63994726,  0.84832379,  0.14964075,  0.90726993,
        -0.69289057,  0.20401277,  0.46849198,  1.4259954 ],
       [ 0.        , -0.84488505, -1.12339636, -0.16054575,  0.53090156,
        -0.69289057, -0.68442195, -0.36506078, -0.19067191],
       [ 1.        ,  1.23388019,  1.94372388, -0.26394125, -1.28821221,
        -0.69289057, -1.10325546,  0.60439732, -0.10558415],
       [ 0.        , -0.84488505, -0.99820778, -0.16054575,  0.15453319,
         0.12330164, -0.49404308, -0.92076261, -1.04154944],
       [ 1.        , -1.14185152,  0.5040552 , -1.50468724,  0.90726993,
         0.76583594,  1.4097456 ,  5.4849091 , -0.0204964 ]])

## Classification via Logistic Regression

Implementation of binary classification using logistic regression for a data set with two classes.
Using ```scipy.optimize.fmin_bfgs``` to optimise your cost function. ```fmin_bfgs``` requires the cost function to be optimised, and the gradient of this cost function. 
Implementation of the function ```train``` that takes a matrix of examples, and a vector of labels, and returns the maximum likelihood weight vector for logistic regresssion. Also implement a function ```test``` that takes this maximum likelihood weight vector and the a matrix of examples, and returns the predictions. See the section **Putting everything together** below for expected usage.

We add an extra column of ones to represent the constant basis.

In [ ]:
data = np.hstack([data, np.ones((data.shape[0], 1))]) # add a column of ones
data[:5,:]

In [4]:
def sigmoid(X):
    """S shaped function, known as the sigmoid"""
    return 1 / (1 + np.exp(- X))

def cost(theta, X, y):
    """The cost function for logistic regression"""
    p_1 = sigmoid(np.dot(X, theta)) # predicted probability of label 1
    log_l = (-y)*np.log(p_1) - (1-y)*np.log(1-p_1) # log-likelihood vector

    return log_l.mean()

def grad(theta, X, y):
    """The gradient of the cost function for logistic regresssion"""
    p_1 = sigmoid(np.dot(X, theta))
    error = p_1 - y # difference between label and prediction
    grad = np.dot(error, X) / y.size # gradient vector

    return grad

def train(X, y):
    """Train a logistic regression model for data X and labels y.
    returns the learned parameter.
    """
    theta = 0.1*np.random.randn(9)
    theta_best = opt.fmin_bfgs(cost, theta, fprime=grad, args=(X, y))
    return theta_best

def predict(theta_best, Xtest):
    """Using the learned parameter theta_best, predict on data Xtest"""
    p = sigmoid(theta_best.dot(Xtest.T))
    for i in range(len(p)):
        if p[i] > 0.5:
            p[i] = 1
        else:
            p[i] = 0
    return p


## Performance measure

There are many ways to compute the [performance of a binary classifier](http://en.wikipedia.org/wiki/Evaluation_of_binary_classifiers). The key concept is the idea of a confusion matrix or contingency table:

|              |    | Label |    |
|:-------------|:--:|:-----:|:--:|
|              |    |  +1   | -1 |
|**Prediction**| +1 |    TP | FP |
|              | -1 |    FN | TN |

where
* TP - true positive
* FP - false positive
* FN - false negative
* TN - true negative

Implemented three functions, the first one which returns the confusion matrix for comparing two lists (one set of predictions, and one set of labels). Then implemented two functions that take the confusion matrix as input and returns the **accuracy** and **balanced accuracy** respectively. Accuracy is defined as the number of correct classifications divided by the total number of examples. The balanced accuracy is the average accuracy of each class, that is the accuracy when the true class is positive and the accuracy when the true class is negative (averaged).


In [5]:
def confusion_matrix(prediction, labels):
    """Returns the confusion matrix for a list of predictions and (correct) labels"""
    assert len(prediction) == len(labels)
    def f(p, l):
        n = 0
        for i in range(len(prediction)):
            if prediction[i] == p and labels[i] == l:
                n += 1
        return n
    return np.matrix([[f(1, 1), f(1, 0)], [f(0, 1), f(0, 0)]])

def confusion_matrix_advanced(prediction, labels):
    """Returns the confusion matrix for a list of predictions and (correct) labels"""
    assert len(prediction) == len(labels)
    f = lambda p, l: len(list(filter(lambda x: x == (p, l), zip(prediction, labels))))
    return np.matrix([[f(1, 1), f(1, 0)], [f(0, 1), f(0, 0)]])

def accuracy(cmatrix):
    """Returns the accuracy of a confusion matrix"""
    tp, fp, fn, tn = cmatrix.flatten().tolist()[0]
    return (tp + tn) / (tp + fp + fn + tn)

def balanced_accuracy(cmatrix):
    """Returns the balanced accuracy of a confusion matrix"""
    tp, fp, fn, tn = cmatrix.flatten().tolist()[0]
    return tp / 2 / (tp + fn) + tn / 2 / (tn + fp)

#M = confusion_matrix([1,1,1,-1,-1,-1],[1,1,-1,1,-1,-1])
#accuracy(M)


## Putting everything together

Consider the following code, which trains on all the examples, and predicts on the training set. Discuss the results.

In [6]:
y = data[:,0]
X = data[:,1:]
theta_best = train(X, y)
pred = predict(theta_best, X)
cmatrix = confusion_matrix(pred, y)
[accuracy(cmatrix), balanced_accuracy(cmatrix)]

Optimization terminated successfully.
         Current function value: 0.470993
         Iterations: 26
         Function evaluations: 27
         Gradient evaluations: 27


[0.7825520833333334, 0.736044776119403]

## Effect of regularization parameter

By splitting the data into two halves, train on one half and report performance on the second half. By repeating this experiment for different values of the regularization parameter $\lambda$ we can get a feeling about the variability in the performance of the classifier due to regularization.$\lambda$. 


In [7]:
### Solution

def split_data(data):
    """Randomly split data into two equal groups"""
    np.random.seed(1)
    N = len(data)
    idx = np.arange(N)
    np.random.shuffle(idx)
    train_idx = idx[:int(N/2)]
    test_idx = idx[int(N/2):]

    X_train = data[train_idx, 1:]
    t_train = data[train_idx, 0]
    X_test = data[test_idx, 1:]
    t_test = data[test_idx, 0]
    
    return X_train, t_train, X_test, t_test

X_train, t_train, X_test, t_test = split_data(data)
